<a href="https://colab.research.google.com/github/jjangmo91/eeSDM_Cervus-nippon/blob/main/eeSDM_Cervus_nippon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **eeSDM_Cervus-nippon**:

In [4]:
!pip install eeSDM

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.3 MB/s eta 0:00:00


In [5]:
import eeSDM
import ee, geemap
import geemap.colormaps as cm

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import random, math, os

import warnings
warnings.filterwarnings("ignore")

In [12]:
# Earth Engine Authentication
ee.Authenticate()

# Initializing Earth Engine
ee.Initialize(project='ee-jjangmo91')

# 1. Add Species Occurrence Data(Local)

# 2. Define Area of Interest (AOI)

In [13]:
# Define Area of Interest (AOI)
# Calculate the bounding box of the Data and expand it by 50,000 meters
AOI = Data.geometry().bounds().buffer(distance=50000, maxError=1000)

# Create an outline image of the AOI for visualization
outline = ee.Image().byte().paint(featureCollection=AOI, color=1, width=3)

# Creating an Empty geemap Map Object
Map = geemap.Map()

Map.addLayer(outline, {'palette': 'FF0000'}, "Study Area")
Map.addLayer(data_raw, {'color': 'blue'}, 'Original data')
Map.addLayer(Data, {'color': 'red'}, 'Final data')
Map.centerObject(Data.geometry(), 6)
Map

NameError: name 'Data' is not defined

# 3. Add Predictive Variables

In [14]:
# WorldClim BIO Variables V1 (1km): A total of 19 climate variables
BIO = ee.Image("WORLDCLIM/V1/BIO")

# NASA SRTM Digital Elevation 30m: Digital elevation data
Terrain = ee.Algorithms.Terrain(ee.Image("USGS/SRTMGL1_003"))

# MOD44B.006 Terra MODIS Vegetation Continuous Fields Yearly Global 250m
# Percent Tree Cover: Median between 2003 and 2020
MODIS = ee.ImageCollection("MODIS/006/MOD44B")
MedianPTC = MODIS.filterDate('2003-01-01', '2020-12-31').select(['Percent_Tree_Cover'], ['PTC']).median()

# Combining Bands into a Single Multi-Band Image
predictors = BIO.addBands(Terrain).addBands(MedianPTC)

# Masking Ocean Pixels & Clipping to the Area of Interest
predictors = predictors.updateMask(ee.Image("USGS/SRTMGL1_003").gt(0)).clip(AOI)

NameError: name 'AOI' is not defined